In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval



import warnings; warnings.simplefilter('ignore')

In [3]:
#Reading the data from csv
md = pd. read_csv('movies_metadata.csv')
md.head()

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview    ...     release_date  \
0  Led by Woody, Andy's toys live happily in his ...    ...       1995-10-30   
1  When siblings Judy and Peter discover an encha...    ...       1995-12-15   
2  A family wedding reignites the ancient feud be...    ...       1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...    ...       1995-12-22   
4  Just when George Banks has recovered from his ...    ...       1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [4]:
#Filling in the null values with data
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [5]:
#calculate mean of vote coounts
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [6]:
#Setting the quantile to further filter the results and displaying the results
m = vote_counts.quantile(0.95)
m

434.0

In [7]:
#Converting the date parameter of the csv file
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
#calculate the votecounts greater the quantile and not zero
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [9]:
##TMDB weighted rating formula
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
#calculate weighted ratring and set it into a column
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [11]:
#Sort the weighted ratings accordingly
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [12]:
#displaying the results
qualified.head(15)

title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   
292                                         Pulp Fiction  1994        8670   
314                             The Shawshank Redemption  1994        8358   
7000       The Lord of the Rings: The Return of the King  2003        8226   
351                                         Forrest Gump  1994        8147   
5814               The Lord of the Rings: The Two Towers  2002        7641   
256                                            Star Wars  1977        6778   
1225                                  Back to the Future  1985        6239   
834                                        The Godfather  1972        6024   
1154                             The Empire Strikes Back  1980        5998   
46                                                 Se7en  1995        5915   

       vote_average popularity  \
15480             8    29.1081   
12481             8    123.167   
22879             8    32.2135   
2843              8    63.8696   
4863              8    32.0707   
292               8     140.95   
314               8    51.6454   
7000              8    29.3244   
351               8    48.3072   
5814              8    29.4235   
256               8    42.1497   
1225              8    25.7785   
834               8    41.1093   
1154              8     19.471   
46                8    18.4574   

                                                  genres        wr  
15480  [Action, Thriller, Science Fiction, Mystery, A...  7.917588  
12481                   [Drama, Action, Crime, Thriller]  7.905871  
22879                [Adventure, Drama, Science Fiction]  7.897107  
2843                                             [Drama]  7.881753  
4863                        [Adventure, Fantasy, Action]  7.871787  
292                                    [Thriller, Crime]  7.868660  
314                                       [Drama, Crime]  7.864000  
7000                        [Adventure, Fantasy, Action]  7.861927  
351                             [Comedy, Drama, Romance]  7.860656  
5814                        [Adventure, Fantasy, Action]  7.851924  
256                 [Adventure, Action, Science Fiction]  7.834205  
1225        [Adventure, Comedy, Science Fiction, Family]  7.820813  
834                                       [Drama, Crime]  7.814847  
1154                [Adventure, Action, Science Fiction]  7.814099  
46                            [Crime, Mystery, Thriller]  7.811669

In [13]:
#Indexing of the genre
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [14]:
#build function for recommending movies making use of the TMDB weighted rating formula and returng the dataframe
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)     [1]
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [15]:
#Thus finally displaying the dataframe depending on the genre thus recommending the highest rated movies in the genre
build_chart('Romance').head(15)

title  year  vote_count  vote_average popularity  \
10309  Dilwale Dulhania Le Jayenge  1995         661             9     34.457   
351                   Forrest Gump  1994        8147             8    48.3072   
876                        Vertigo  1958        1162             8    18.2082   
40251                   Your Name.  2016        1030             8  34.461252   
883               Some Like It Hot  1959         835             8    11.8451   
1132               Cinema Paradiso  1988         834             8     14.177   
19901                     Paperman  2012         734             8    7.19863   
37863                  Sing Street  2016         669             8  10.672862   
882                  The Apartment  1960         498             8    11.9943   
38718               The Handmaiden  2016         453             8  16.727405   
3189                   City Lights  1931         444             8    10.8915   
24886             The Way He Looks  2014         262             8    5.71127   
45437               In a Heartbeat  2017         146             8   20.82178   
1639                       Titanic  1997        7770             7    26.8891   
19731      Silver Linings Playbook  2012        4840             7    14.4881   

             wr  
10309  8.565285  
351    7.971357  
876    7.811667  
40251  7.789489  
883    7.745154  
1132   7.744878  
19901  7.713951  
37863  7.689483  
882    7.599317  
38718  7.566166  
3189   7.558867  
24886  7.331363  
45437  7.003959  
1639   6.981546  
19731  6.970581

#Thus the motive behind bThe text in the document by <Romell Segaran,Akshay Singh,Sushant Dhar> is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

The code in the document by <Romell Segaran,Akshay Singh,Sushant Dhar> is licensed under the MIT License https://opensource.org/licenses/MITuilding the simple recommender is making use of a formula known as the TMDB formula for calculating the weighted 
#rating of each mpovie and thus sorting them according to the weighted ratings according to the selction of the user


#Weighted Rating Formula
weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
Where:
R = average for the movie (mean) = (Rating)
v = number of votes for the movie = (votes)
m = minimum votes required to be listed in the Top 250 (currently 25000)
C = the mean vote across the whole report (currently 7.0)

#Referneces
1)(https://www.quora.com/How-does-IMDbs-rating-system-work)

#Drawbacks 
1)The model thus not take into use the choice of the user and just recoommends the user results depending on their calculated formula

#Advantages
1)The weighted rating of movis can be easily calculated and thus new movies can be easily added

#Improvements
1)Making use of content based type filtering that makes use of the users choices and provides results accordingly
2)Further improving the model making use of collaborative filtering which takes into account the relationship between the users




The text in the document by <Romell Segaran,Akshay Singh,Sushant Dhar> is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

The code in the document by <Romell Segaran,Akshay Singh,Sushant Dhar> is licensed under the MIT License https://opensource.org/licenses/MIT